In [2]:
# Get the directory of the current notebook
notebook_dir = os.path.dirname(os.path.abspath('setup.ipynb'))

# Get the project root (parent directory)
project_root = os.path.dirname(notebook_dir)
sys.path.append(project_root)

In [7]:
from dotenv import load_dotenv
import os
import sys
from google.cloud import bigquery
import pandas as pd
from my_utils.utils import test_function

load_dotenv()  # Loads variables from .env into os.environ

credentials_path = os.getenv("GOOGLE_APPLICATION_CREDENTIALS") # Gets the value of GOOGLE_APPLICATION_CREDENTIALS from the environment

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path # Explicitly sets (or ensures) the environment variable inside Python's os.environ


In [8]:
# Initialize BigQuery client
client = bigquery.Client()

In [9]:
WITH cephalosporin_resistant AS (
  SELECT subject_id, chartdate
  FROM `physionet-data.mimiciv_3_1_hosp.microbiologyevents`
  WHERE interpretation = 'R'
    AND LOWER(ab_name) LIKE '%cef%'
),

carbapenem_susceptible AS (
  SELECT subject_id, chartdate
  FROM `physionet-data.mimiciv_3_1_hosp.microbiologyevents`
  WHERE interpretation = 'S'
    AND LOWER(ab_name) LIKE '%enem%'
    AND LOWER(ab_name) NOT LIKE '%actam%' -- exclude combination therapies
),

-- Combine the above to identify ESBL profiles
esbl_profile AS (
  SELECT DISTINCT ceph.subject_id, ceph.chartdate
  FROM cephalosporin_resistant ceph
  INNER JOIN carbapenem_susceptible carb
    ON ceph.subject_id = carb.subject_id
   AND ceph.chartdate = carb.chartdate
),

-- filter for non-ESBL EKP samples
non_esbl_ekp_samples AS (
  SELECT DISTINCT subject_id, chartdate
  FROM `physionet-data.mimiciv_3_1_hosp.microbiologyevents`
  WHERE (
    LOWER(org_name) LIKE '%escherichia coli%'
    OR LOWER(org_name) LIKE '%klebsiella pneumoniae%'
    OR LOWER(org_name) LIKE '%proteus mirabilis%'
  )
  EXCEPT DISTINCT
  SELECT * FROM esbl_profile
)

-- Pull all antibiogram entries for those non-ESBL EKP samples
SELECT patho.*
FROM `physionet-data.mimiciv_3_1_hosp.microbiologyevents` AS patho
JOIN non_esbl_ekp_samples AS non_esbl
  ON patho.subject_id = non_esbl.subject_id
 AND patho.chartdate = non_esbl.chartdate
WHERE (
  LOWER(patho.org_name) LIKE '%escherichia coli%'
  OR LOWER(patho.org_name) LIKE '%klebsiella pneumoniae%'
  OR LOWER(patho.org_name) LIKE '%proteus mirabilis%'
)
AND patho.interpretation IS NOT NULL
ORDER BY patho.subject_id, patho.chartdate;

IndentationError: unindent does not match any outer indentation level (<string>, line 43)

In [7]:
query = """
    WITH cephalosporin_resistant AS (
  SELECT subject_id, chartdate
  FROM `physionet-data.mimiciv_3_1_hosp.microbiologyevents`
  WHERE interpretation = 'R'
    AND LOWER(ab_name) LIKE '%cef%'
),

carbapenem_susceptible AS (
  SELECT subject_id, chartdate
  FROM `physionet-data.mimiciv_3_1_hosp.microbiologyevents`
  WHERE interpretation = 'S'
    AND LOWER(ab_name) LIKE '%enem%'
    AND LOWER(ab_name) NOT LIKE '%actam%' -- exclude combination therapies
),

-- Combine the above to identify ESBL profiles
esbl_profile AS (
  SELECT DISTINCT ceph.subject_id, ceph.chartdate
  FROM cephalosporin_resistant ceph
  INNER JOIN carbapenem_susceptible carb
    ON ceph.subject_id = carb.subject_id
   AND ceph.chartdate = carb.chartdate
),

-- filter for non-ESBL EKP samples
non_esbl_ekp_samples AS (
  SELECT DISTINCT subject_id, chartdate
  FROM `physionet-data.mimiciv_3_1_hosp.microbiologyevents`
  WHERE (
    LOWER(org_name) LIKE '%escherichia coli%'
    OR LOWER(org_name) LIKE '%klebsiella pneumoniae%'
    OR LOWER(org_name) LIKE '%proteus mirabilis%'
  )
  EXCEPT DISTINCT
  SELECT * FROM esbl_profile
)

-- Pull all antibiogram entries for those non-ESBL EKP samples
SELECT patho.*
FROM `physionet-data.mimiciv_3_1_hosp.microbiologyevents` AS patho
JOIN non_esbl_ekp_samples AS non_esbl
  ON patho.subject_id = non_esbl.subject_id
 AND patho.chartdate = non_esbl.chartdate
WHERE (
  LOWER(patho.org_name) LIKE '%escherichia coli%'
  OR LOWER(patho.org_name) LIKE '%klebsiella pneumoniae%'
  OR LOWER(patho.org_name) LIKE '%proteus mirabilis%'
)
AND patho.interpretation IS NOT NULL
ORDER BY patho.subject_id, patho.chartdate;

"""

query_job = client.query(query)

query_job.result()

df = query_job.result().to_dataframe()

print(df.head())


# # Set up parameters
# job_config = bigquery.QueryJobConfig(
#     query_parameters=[
#         bigquery.ScalarQueryParameter("subject_id", "INT64", subject_id)
#     ]
# )

# # Run query
# query_job = client.query(query, job_config=job_config)
# df = query_job.result().to_dataframe()

# print(df.head())

Forbidden: 403 Access Denied: Table physionet-data:mimiciv_3_1_hosp.microbiologyevents: User does not have permission to query table physionet-data:mimiciv_3_1_hosp.microbiologyevents, or perhaps it does not exist.; reason: accessDenied, message: Access Denied: Table physionet-data:mimiciv_3_1_hosp.microbiologyevents: User does not have permission to query table physionet-data:mimiciv_3_1_hosp.microbiologyevents, or perhaps it does not exist.

Location: US
Job ID: 0d016b82-58a3-4cf0-9515-cd223768ad54
